<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/intervviewZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
#!pip install git+https://github.com/openai/whisper.git
!pip install -U git+https://github.com/jianfch/stable-ts.git

  Cloning https://github.com/jianfch/stable-ts.git to /tmp/pip-req-build-7ce31j8h
  Running command git clone --filter=blob:none --quiet https://github.com/jianfch/stable-ts.git /tmp/pip-req-build-7ce31j8h
  Resolved https://github.com/jianfch/stable-ts.git to commit 7dab1714c5915e3efe070ae697ade1d04f5c8c42
  Preparing metadata (setup.py) ... done


In [157]:
!pip install moviepy

In [158]:
!pip install ffmpeg-python

In [159]:
#import whisper
import stable_whisper
import ffmpeg
import numpy as np
import json, re, os
import glob
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from moviepy.editor import VideoFileClip
from moviepy.editor import AudioFileClip

In [160]:
!cp ./drive/MyDrive/IZ-2218確認４/*.* ./

In [161]:
def is_webm(path):
    with open('./' + path) as f:
        for _line in f:
            if _line.rstrip()[-5:] == '.webm':
                return True
    return False

In [162]:
def conv_webm(input_file):
    #!ffmpeg -allowed_extensions ALL -f hls -i ./drive/MyDrive/IZ-2218確認３/ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.m3u8 -c:v copy -y ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm
    webm_tmp = "./video_tmp.webm"
    try:
        (
            ffmpeg
            .input('./' + input_file, allowed_extensions='ALL')
            .output(webm_tmp, vcodec="copy")
            .overwrite_output()
            .run(capture_stderr=True, capture_stdout=True)
        )
        _clip = VideoFileClip(webm_tmp)
    except ffmpeg.Error as e:
        #print(f"{e.stderr.decode()}")
        #print(f"標準出力: {e.stdout.decode()}")
        _clip = None
    return _clip

In [163]:
def get_video_list(path):
    video_path = f'{path}*_video.m3u8'
    video_paths = glob.glob(video_path)
    video_paths.sort()
    return video_paths
def get_audio_list(path):
    audio_path = f'{path}*_audio.m3u8'
    audio_paths = glob.glob(audio_path)
    audio_paths.sort()
    return audio_paths

In [164]:
path = './'
video_paths = get_video_list(path)
audio_paths = get_audio_list(path)

In [165]:
video = []
for video_path in video_paths:
    _video = conv_webm(video_path) if is_webm(video_path) else VideoFileClip(video_path)
    video.append(_video)
print(video)

[<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7813903534f0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x781390435d20>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7813904edd20>]


In [166]:
audio = []
wav_paths = []
for audio_path in audio_paths:
    wav_path = '.'.join(audio_path.split('.')[:-1]) + '.wav'
    audio_tmp = AudioFileClip(audio_path)
    audio_tmp.write_audiofile(wav_path, ffmpeg_params=["-ac", "1"])
    audio.append(audio_tmp)
    wav_paths.append(wav_path)
print(audio)

MoviePy - Writing audio in ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_186868730__uid_e_audio.wav


MoviePy - Done.
MoviePy - Writing audio in ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_801600513__uid_e_audio.wav


MoviePy - Done.
MoviePy - Writing audio in ./ece061b98f490713b71deabf1c7df387_1731585061833__uid_s_846732376__uid_e_audio.wav


MoviePy - Done.
[<moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x781377f984f0>, <moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x78137c1df5e0>, <moviepy.audio.io.AudioFileClip.AudioFileClip object at 0x7813904eeb90>]


In [167]:
#model = whisper.load_model("large")
model = stable_whisper.load_model('large')
stable_whisper.modify_model(model)

  checkpoint = torch.load(fp, map_location=device)



OutOfMemoryError: CUDA out of memory. Tried to allocate 254.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 35.06 MiB is free. Process 17269 has 14.71 GiB memory in use. Of the allocated memory 14.20 GiB is allocated by PyTorch, and 396.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
asr_result = []
for wav_patn in wav_paths:
    print(f'\n---- {wav_patn} ----')
    _result = model.transcribe(wav_patn, verbose=True, fp16=False, language="ja", word_timestamps=True)
    asr_result.append(_result)

In [ ]:
SEG = []
for spk, res in enumerate(asr_result):
    for seg in res.segments:
        SEG.append([seg.start, seg.end, seg.text, f'speaker_{format(spk, "02") }'])

In [ ]:
SEG_SORTED = sorted(SEG, key=lambda x:(x[0], x[1]))
SEG_SORTED


In [ ]:
SEG_SORTED

In [ ]:
print(int)

In [ ]:
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか(?!ね)|でしょうか(?!ね)|ましたか|されましたか|いつ頃')
INTERVIEWERE = {}
INT_COUNT = []
for c in range(len(asr_result)):
    INT_COUNT.append([0, 0, 0, ''])
for srt, end, txt, spk in SEG_SORTED:
    count = len(prog.findall(txt))
    spk_no = int(spk.split('_')[-1])
    INT_COUNT[spk_no][0] += 1
    INT_COUNT[spk_no][3] = spk
    if count > 0:
        INT_COUNT[spk_no][1] += count
for c, cnt in enumerate(INT_COUNT):
    INT_COUNT[c][2] = cnt[1] / cnt[0]
INT_COUNT.index(min(INT_COUNT))

rate = [c[2] for c in INT_COUNT]
min_index = rate.index(min(rate))

for i, intv in enumerate(INT_COUNT):
    if i == min_index:
        INTERVIEWERE[intv[3]] = "モニター"
    else:
        INTERVIEWERE[intv[3]] = "インタビューア" + intv[3].split('_')[-1]
print(INTERVIEWERE)

In [ ]:
#INT_COUNT = [[0] * 3] * len(asr_result)
#INT_COUNT = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
#INT_COUNT = [[0, 0, 0]] * len(asr_result)
INT_COUNT = []
for c in range(len(asr_result)):
    INT_COUNT.append([0, 0, 0])
INT_COUNT[2][2] = 1
print(INT_COUNT[0][0])
print(INT_COUNT[0][1])
print(INT_COUNT[0][2])
print(INT_COUNT[1][0])
print(INT_COUNT[1][1])
print(INT_COUNT[1][2])
print(INT_COUNT[2][0])
print(INT_COUNT[2][1])
print(INT_COUNT[2][2])

0
0
0
0
0
0
0
0
1


In [ ]:
whisper_result = {}
whisper_segment = []
for asr in asr_result['segments']:
    if asr['end'] - asr['start'] >= 0.6 and len(asr['text']) >= 2:
        asr['text'] += '。'
        whisper_segment.append(asr)
whisper_result['segments'] = whisper_segment

In [ ]:
whisper_result

In [ ]:
#speaker_count = 2
speaker_count = 3

import torch
pipeline.to(torch.device("cuda"))
diarization_result = pipeline(audio_path, num_speakers=speaker_count)
#diarization_result = pipeline(audio_path, min_speakers=2, max_speakers=5)

In [ ]:
diarization_result

In [ ]:
#final_result = diarize_text(asr_result, diarization_result)
final_result = diarize_text(whisper_result, diarization_result)

In [ ]:
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
INT_COUNT = []
for seg, spk, sent in final_result:
  if spk is None:
    spk = INTERVIEW[-1]['spk']
  count = len(prog.findall(sent))
  spk_no = int(spk.split('_')[-1])
  if count > 0: print(str(sent))
  _seg = {'start': round(seg.start, 2), 'end': round(seg.end, 2), 'text': sent, 'count': count, 'spk': spk, 'spk_no': spk_no}
  INTERVIEW.append(_seg)
  INT_COUNT.append(int(spk[-1]))

speaker_count = max(INT_COUNT) + 1

speek = [0] * speaker_count
for INTV in INTERVIEW:
    speek[INTV["spk_no"]] += INTV["count"]
inteviewer_index = np.argmax(speek)
monitor_index = np.argmin(speek)

print(speek)
print(f'インタービューア ： SPEAKER_0{inteviewer_index}')
print(f'モニター　　　　 ： SPEAKER_0{monitor_index}')

In [ ]:
for seg, spk, sent in final_result:
    speeker_name = spk
    #speeker_name =   'モニタ　　　　'
    #if spk == 'SPEAKER_0' + str(inteviewer_index):
    #  speeker_name = 'インタビューア'
    line = f'{seg.start:.2f}s~{seg.end:.2f}s {speeker_name} {sent}'
    print(line)

In [ ]:
InterReview_json = {"movie_name": video_name}
speak_json = []
for seg, spk, sent in final_result:
    _speak = {"start": f'{seg.start:.2f}s', "end": f'{seg.end:.2f}s', 'speaker': spk, 'speak': sent}
    speak_json.append(_speak)
InterReview_json['InterReview'] = speak_json

json_path = '.'.join(video_path.split('.')[:-1]) + '.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(InterReview_json, f, indent=4, ensure_ascii=False)


In [ ]:
!ls -alh ./

In [ ]:
!cp ./InterviewZero_0_audio_mask.json ./drive/MyDrive

In [ ]:
!cp ./InterviewZero_1_audio_mask.json ./drive/MyDrive

In [ ]:
!cp ./InterviewZero_2_audio_mask.json ./drive/MyDrive